# Image Endpoint

Creates HTTP endpoint to accept images and call a model

In [ ]:
import os
"""
os.environ['create_image']='True'
os.environ['repository']='romeokienzler'
os.environ['version']='0.10'
"""
#os.environ['install_requirements']='True'

In [4]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM ultralytics/yolov5:latest
    RUN pip install ipython nbformat flask
    ADD image-endpoint.ipynb /
    ENTRYPOINT ["ipython","/image-endpoint.ipynb","> /tmp/component.log","2> /tmp/component.err"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t claimed-predict-image-endpoint:`echo $version` .
    !docker tag claimed-predict-image-endpoint:`echo $version` `echo $repository`/claimed-predict-image-endpoint:`echo $version`
    !docker push `echo $repository`/claimed-predict-image-endpoint:`echo $version`
elif bool(os.environ.get('install_requirements',False)):
    !pip install flask

Sending build context to Docker daemon  15.22MB
Step 1/4 : FROM ultralytics/yolov5:latest
 ---> bd6f40b6f720
Step 2/4 : RUN pip install ipython nbformat flask
 ---> Using cache
 ---> 014f071df204
Step 3/4 : ADD image-endpoint.ipynb /
 ---> ab502ab44aa7
Step 4/4 : ENTRYPOINT ["ipython","/image-endpoint.ipynb","> /tmp/component.log","2> /tmp/component.err"]
 ---> Running in d654b4038ee8
Removing intermediate container d654b4038ee8
 ---> 976f02007881
Successfully built 976f02007881
Successfully tagged claimed-predict-image-endpoint:0.10
The push refers to repository [docker.io/romeokienzler/claimed-predict-image-endpoint]

e0ec5bdb: Preparing 
6b200e01: Preparing 
0723ec63: Preparing 
0a2f6757: Preparing 
bf18a086: Preparing 
4d267321: Preparing 
07e65d60: Preparing 
3217474a: Preparing 
931a8d95: Preparing 
b251c799: Preparing 
4a9ef494: Preparing 
2723dd5e: Preparing 
293b5ba9: Preparing 
0b612187: Preparing 
8d50284b: Preparing 
8e91ad93: Preparing 
918f9673: Preparing 
31a8d95: Waitin

In [ ]:
from flask import Flask
from flask import request
from flask import send_file

In [ ]:
index_html = """
<html>
    <head></head>
    <body>
        <form action="process-image" method="post" enctype="multipart/form-data">
            <input type="file" name="file" id="file">
            <input type="submit" value="Upload Image" name="submit">
        </form>
    </body>
</html>
"""

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return index_html


@app.route('/process-image', methods=['POST'])
def upload_file():
    file = request.files['file']
    file.save('/tmp/image.png')
    !rm -Rf /tmp/exp
    !python /usr/src/app/detect.py --save-txt --project /tmp/ --source /tmp/image.png
    return send_file('/tmp/exp/image.png', mimetype='image/png')


app.run(host='0.0.0.0', port=8080)